In [ ]:
import torch
from torch import nn # 모델 컴포넌트를 위한 모듈
from torch.utils.data import DataLoader # 데이터로더
from torchvision import datasets # 컴퓨터비전 관련 데이터셋을 위한 모듈
from torchvision.transforms import ToTensor # 데이터셋 변환을 위한 모듈

In [ ]:
train_dataset = datasets.MNIST(
    root="data", # 경로
    train=True, # 학습 데이터셋인지, 검증 데이터셋인지
    download=True, # 다운로드 여부
    transform=ToTensor(), # 이미지 변환
)

val_dataset = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size) # 학습 데이터로더
val_dataloader = DataLoader(val_dataset, batch_size=batch_size) # 검증 데이터로더

for inputs, labels in train_dataloader:
    print(f"Shape of inputs [N, C, H, W]: {inputs.shape}")
    print(f"Shape of labels: {labels.shape} {labels.dtype}")
    break

Shape of inputs [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of labels: torch.Size([64]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # 배치 차원을 제외한 모든 차원을 1차원으로 바꾸어준다.
        self.linear_relu_stack = nn.Sequential( # 심층 신경망 (이해할 필요 없음)
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device) # 모델 인스턴스를 만들고 device로 보낸다.
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss() # 크로스 엔트로피 함수
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Stochastic Gradient Descent

In [ ]:
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset) # 데이터셋 내 샘플 수

    model.train() # "학습 모드"로 설정

    for batch, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device) # 데이터를 device로 보낸다.
    
        outputs = model(inputs) # forward 연산

        loss = criterion(outputs, labels) # 손실 계산

        optimizer.zero_grad() # 기울기 초기화
        loss.backward() # 기울기 계산
        optimizer.step() # 모델 파라미터 업데이트

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(inputs)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def eval(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # "검증 모드"로 설정
    eval_loss, correct = 0, 0
    with torch.no_grad(): # 기울기를 계산하지 않도록 설정
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            eval_loss += criterion(outputs, labels).item()
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
    eval_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg Loss: {eval_loss:>8f} \n")

In [ ]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n--------------------")
    train(train_dataloader, model, criterion, optimizer)
    eval(val_dataloader, model, criterion)
print("Done!")

Epoch 1
--------------------
loss: 2.307070 [    0/60000]
loss: 2.294226 [ 6400/60000]
loss: 2.298607 [12800/60000]
loss: 2.287977 [19200/60000]
loss: 2.296165 [25600/60000]
loss: 2.279234 [32000/60000]
loss: 2.259966 [38400/60000]
loss: 2.281284 [44800/60000]
loss: 2.261573 [51200/60000]
loss: 2.249815 [57600/60000]
Test Error: 
 Accuracy: 40.8%, Avg Loss: 2.256855 

Epoch 2
--------------------
loss: 2.257752 [    0/60000]
loss: 2.245417 [ 6400/60000]
loss: 2.259787 [12800/60000]
loss: 2.224920 [19200/60000]
loss: 2.244625 [25600/60000]
loss: 2.230281 [32000/60000]
loss: 2.198887 [38400/60000]
loss: 2.233263 [44800/60000]
loss: 2.199100 [51200/60000]
loss: 2.184335 [57600/60000]
Test Error: 
 Accuracy: 51.7%, Avg Loss: 2.190133 

Epoch 3
--------------------
loss: 2.189850 [    0/60000]
loss: 2.174104 [ 6400/60000]
loss: 2.201178 [12800/60000]
loss: 2.131346 [19200/60000]
loss: 2.166208 [25600/60000]
loss: 2.152397 [32000/60000]
loss: 2.099649 [38400/60000]
loss: 2.152148 [44800/6000